In [ ]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout


Found 5712 images belonging to 4 classes.
Found 1311 images belonging to 4 classes.


In [ ]:

# STEP 1: DEFINE DATASET PATHS
dataset_path = r"C:\Users\jones\OneDrive\Desktop\Brain_Tumor_Detection"
train_dir = os.path.join(dataset_path, "Training")
test_dir = os.path.join(dataset_path, "Testing")
categories = ["glioma", "meningioma", "notumor", "pituitary"]


In [ ]:

# STEP 2: SET IMAGE SIZE AND BATCH SIZE
image_size = (150, 150)
batch_size = 32
epochs = 50


In [ ]:

# STEP 3: DATA AUGMENTATION
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode="nearest"
)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode="categorical"
)

test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode="categorical",
    shuffle=False
)


In [ ]:

# STEP 4: BUILD THE CNN MODEL
model = Sequential([
    Conv2D(32, (3, 3), activation="relu", input_shape=(image_size[0], image_size[1], 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation="relu"),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation="relu"),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation="relu"),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(512, activation="relu"),
    Dropout(0.5),
    Dense(len(categories), activation="softmax")
])

# Compile the model
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])


In [21]:

# STEP 5: TRAIN THE MODEL
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=epochs,
    validation_data=test_generator,
    validation_steps=test_generator.samples // batch_size
)


Epoch 1/50
178/178 ━━━━━━━━━━━━━━━━━━━━ 121s 668ms/step - accuracy: 0.4782 - loss: 1.1168 - val_accuracy: 0.6242 - val_loss: 0.9684
Epoch 2/50
178/178 ━━━━━━━━━━━━━━━━━━━━ 1s 802us/step - accuracy: 0.6250 - loss: 0.9514 - val_accuracy: 0.9032 - val_loss: 0.5886
Epoch 3/50
178/178 ━━━━━━━━━━━━━━━━━━━━ 106s 596ms/step - accuracy: 0.6870 - loss: 0.7874 - val_accuracy: 0.6883 - val_loss: 0.8371
Epoch 4/50
178/178 ━━━━━━━━━━━━━━━━━━━━ 1s 998us/step - accuracy: 0.8438 - loss: 0.3943 - val_accuracy: 0.9355 - val_loss: 0.4778
Epoch 5/50
178/178 ━━━━━━━━━━━━━━━━━━━━ 128s 715ms/step - accuracy: 0.7311 - loss: 0.6881 - val_accuracy: 0.6531 - val_loss: 0.9292
Epoch 6/50
178/178 ━━━━━━━━━━━━━━━━━━━━ 1s 682us/step - accuracy: 0.7500 - loss: 0.7468 - val_accuracy: 0.6774 - val_loss: 0.9111
Epoch 7/50
178/178 ━━━━━━━━━━━━━━━━━━━━ 105s 591ms/step - accuracy: 0.7876 - loss: 0.5548 - val_accuracy: 0.6234 - val_loss: 1.1174
Epoch 8/50
178/178 ━━━━━━━━━━━━━━━━━━━━ 1s 631us/step - accuracy: 0.8750 - loss: 0

In [22]:

# STEP 6: EVALUATE THE MODEL
loss, accuracy = model.evaluate(test_generator, steps=test_generator.samples // batch_size)
print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")


40/40 ━━━━━━━━━━━━━━━━━━━━ 6s 145ms/step - accuracy: 0.9067 - loss: 0.2633
Test Loss: 0.1970
Test Accuracy: 0.9336


In [23]:

# STEP 7: CONFUSION MATRIX
predictions = model.predict(test_generator)
predicted_categories = np.argmax(predictions, axis=1)
true_categories = test_generator.classes

confusion_matrix = tf.math.confusion_matrix(true_categories, predicted_categories)


41/41 ━━━━━━━━━━━━━━━━━━━━ 6s 148ms/step


In [24]:

# STEP 8: CALCULATE PRECISION, RECALL, AND F1-SCORE
def calculate_metrics(cm):
    precision = np.diag(cm) / np.sum(cm, axis=0)
    recall = np.diag(cm) / np.sum(cm, axis=1)
    f1_score = 2 * (precision * recall) / (precision + recall)
    return precision, recall, f1_score

precision, recall, f1_score = calculate_metrics(confusion_matrix.numpy())
for i, category in enumerate(categories):
    print(f"Class: {category}")
    print(f"Precision: {precision[i]:.4f}")
    print(f"Recall: {recall[i]:.4f}")
    print(f"F1-Score: {f1_score[i]:.4f}\n")


Class: glioma
Precision: 0.9677
Recall: 0.9000
F1-Score: 0.9326

Class: meningioma
Precision: 0.8986
Recall: 0.8399
F1-Score: 0.8682

Class: notumor
Precision: 0.9176
Recall: 0.9901
F1-Score: 0.9525

Class: pituitary
Precision: 0.9612
Recall: 0.9900
F1-Score: 0.9754



In [25]:

# STEP 9: SAVE THE MODEL
model.save("Brain_tumor_detection_model.keras")
